In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import re
import pickle
import gc
import nltk
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.0.0


In [5]:
data = pd.read_csv("./data/train.csv")
testdata = pd.read_csv("./data/test.csv")
data.dropna(how="any", inplace=True)

In [6]:
def cleaner(sent):
    #Two tokens (WEBSITE, VULGAR) are created and punctuation is spaced out
    sent = " ".join(np.vectorize(lambda s:"WEBSITE" if "http" in s else s)(np.array(sent.split())))
    sent = re.sub("[!.?:;,`]", " ", sent)
    return re.sub(re.compile('(?:\*){2,}'),"VULGAR",sent).lower()
for col in ['text','selected_text']:
    data[col] = data[col].apply(lambda s: cleaner(s))
testdata['text'] = testdata['text'].apply(lambda s: cleaner(s))
data = data.loc[data.text.apply(lambda s: len(s))!=1]

In [8]:
PATH = './tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)

<h2> N-Gram Level Subtext Model </h2>

We continue with the observations established in ATTEMPT 2 regarding the similarity between "text" and "selected_text" for neutral sentiments which makes it favorable to merely predict the entire text (at 0.92 accuracy). For positive and negative labels, we attempt classifier and regressor designs to predict subtexts, or predict Jaccardian similarity with subtexts.

<b> STEP 1: Classifier </b>

In [ ]:
def X(text):
    ngrams, text = [], text.split()
    for i in range(len(text)):
        for j in range(i, len(text)):
            ngrams.append(" ".join(text[i:j+1]))
    return np.array(ngrams)

<b>STEP 2: Regressor </b>

In [ ]:
def Y(ngrams, subset):
    def jaccard(str1, str2):
        a, b = set(str1.lower().split()), set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    return np.vectorize(lambda s: jaccard(s, subset))(np.array(ngrams))

<h2> Q/A Attention Design Model </h2>